In [1]:
import os

from groq import Groq

import json

import base64

import cv2

Capture Image

In [2]:
# List to store clicked coordinates
Coords = []

# Callback function to handle mouse click
def mouse_click(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f"Clicked at pixel coordinates: x = {x}, y = {y}")
        Coords.append((x, y))
        cv2.imwrite("apiimg.jpeg", param['frame'])
        print("Image saved as 'apiimg.jpeg'.")

# Initialize webcam
cap = cv2.VideoCapture(1)  # Change camera index if needed

# Set mouse callback
cv2.namedWindow("Webcam Feed")

print("Click on the object in the video window to get pixel coordinates and save the image.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame from webcam.")
        break

    # Set mouse callback on every fresh frame
    cv2.setMouseCallback("Webcam Feed", mouse_click, param={'frame': frame.copy()})

    cv2.imshow("Webcam Feed", frame)

    key = cv2.waitKey(1)
    if key == 27:  # ESC to exit
        break

cap.release()
cv2.destroyAllWindows()


Click on the object in the video window to get pixel coordinates and save the image.
Clicked at pixel coordinates: x = 1078, y = 405
Image saved as 'apiimg.jpeg'.


In [3]:
prompt = """
Detect and return the centroid coordinates (in pixels) for each colored cube in the image.

Please return the results in the following JSON format:

[
  {
    "color": "box1",
    "centroid": {
      "x": 145,
      "y": 105
    }
  },
  {
    "color": "box2",
    "centroid": {
      "x": 224,
      "y": 156
    }
  }
]

Only respond with the JSON and no additional explanation.
"""

Encode images

In [4]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [5]:
image_path = "apiimg.jpeg"

In [6]:
base64_image = encode_image(image_path)

In [7]:
client = Groq(
    api_key="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"
)

completion = client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                    }
                }
            ]
        }
    ],
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
    stream=False,
    stop=None,
)

with open("output.json", "w") as outfile:
    json.dump(completion.model_dump(), outfile, indent=2)

# Also print just the assistant's reply
print(completion.choices[0].message.content)

[
 {
 "color": "red",
 "centroid": {
 "x":532,
 "y":171
 }
 },
 {
 "color": "yellow",
 "centroid": {
 "x":600,
 "y":173
 }
 },
 {
 "color": "blue",
 "centroid": {
 "x":465,
 "y":275
 }
 },
 {
 "color": "green",
 "centroid": {
 "x":516,
 "y":384
 }
 }
]


export GROQ_API_KEY=<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>
